In [2]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from keras.optimizers import SGD
import math
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle

Using TensorFlow backend.


In [3]:
import multiprocessing
import gensim
import nltk
from collections import defaultdict
from gensim.test.utils import datapath
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases,Phraser

In [86]:
import pandas as pd

data = pd.read_csv('preprocessed_data.csv',index_col=None,header=0)
data['lemmatized'][525] = ''
data = shuffle(data)
msk = np.random.rand(len(data)) < 0.8
data.reset_index(inplace=True,drop=True)
train = data[msk]
test = data[~msk]
len(test)

c:\users\limri\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


249

In [9]:
import os
class MySentences(object):
    """MySentences is a generator to produce a list of tokenized sentences 
    
    Takes a list of numpy arrays containing documents.
    
    Args:
        arrays: List of arrays, where each element in the array contains a document.
    """
    def __init__(self, *arrays):
        self.arrays = arrays
 
    def __iter__(self):
        for array in self.arrays:
            for document in array:
                for sent in nltk.sent_tokenize(document):
                    yield nltk.word_tokenize(sent)

def get_word2vec(sentences, location):
    """Returns trained word2vec
    
    Args:
        sentences: iterator for sentences
        
        location (str): Path to save/load word2vec
    """
    if os.path.exists(location):
        print('Found {}'.format(location))
        model = gensim.models.Word2Vec.load(location)
        return model
    
    print('{} not found. training model'.format(location))
    model = gensim.models.Word2Vec(sentences, size=100, window=5, min_count=5, workers=4)
    print('Model done training. Saving to disk')
    model.save(location)
    return model

In [11]:
w2vec = get_word2vec(
    MySentences(
        train['lemmatized'].values, 
        #df_test['Text'].values  Commented for Kaggle limits
    ),
    'w2vmodel'
)

w2vmodel not found. training model


W0718 18:19:55.844675 11872 base_any2vec.py:1386] under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay


Model done training. Saving to disk


In [16]:
class MyTokenizer:
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        transformed_X = []
        for document in X:
            tokenized_doc = []
            for sent in nltk.sent_tokenize(document):
                tokenized_doc += nltk.word_tokenize(sent)
            transformed_X.append(np.array(tokenized_doc))
        return np.array(transformed_X)
    
    def fit_transform(self, X, y=None):
        return self.transform(X)

class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(word2vec.wv.syn0[0])

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = MyTokenizer().fit_transform(X)
        
        return np.array([
            np.mean([self.word2vec.wv[w] for w in words if w in self.word2vec.wv]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
    
    def fit_transform(self, X, y=None):
        return self.transform(X)

In [17]:
mean_embedding_vectorizer = MeanEmbeddingVectorizer(w2vec)
mean_embedded = mean_embedding_vectorizer.fit_transform(train['lemmatized'])

c:\users\limri\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).


In [18]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.optimizers import Adam

# Use the Keras tokenizer
num_words = 2000
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(train['lemmatized'].values)

In [19]:
X = tokenizer.texts_to_sequences(train['lemmatized'].values)
X = pad_sequences(X, maxlen=2000)

In [20]:
# Build out our simple LSTM
embed_dim = 128
lstm_out = 196

# Model saving callback
ckpt_callback = ModelCheckpoint('keras_model', 
                                 monitor='val_loss', 
                                 verbose=1, 
                                 save_best_only=True, 
                                 mode='auto')

model = Sequential()
model.add(Embedding(num_words, embed_dim, input_length = X.shape[1]))
model.add(LSTM(lstm_out, recurrent_dropout=0.2, dropout=0.2))
model.add(Dense(4,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['categorical_crossentropy'])
print(model.summary())

W0718 18:21:52.244302 11872 deprecation_wrapper.py:119] From c:\users\limri\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0718 18:21:52.297375 11872 deprecation_wrapper.py:119] From c:\users\limri\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0718 18:21:52.308733 11872 deprecation_wrapper.py:119] From c:\users\limri\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0718 18:21:52.506720 11872 deprecation_wrapper.py:119] From c:\users\limri\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is d

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2000, 128)         256000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 788       
Total params: 511,588
Trainable params: 511,588
Non-trainable params: 0
_________________________________________________________________
None


In [57]:
from sklearn.model_selection import train_test_split

In [69]:
Y = pd.get_dummies(train['labels']).values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42, stratify=Y)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(795, 2000) (795, 4)
(199, 2000) (199, 4)


In [23]:
batch_size = 32
model.fit(X_train, Y_train, epochs=8, batch_size=batch_size, validation_split=0.2, callbacks=[ckpt_callback])

W0718 18:22:06.310514 11872 deprecation.py:323] From c:\users\limri\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 636 samples, validate on 159 samples
Epoch 1/8
636/636 [==============================] - 121s 191ms/step - loss: 1.2268 - categorical_crossentropy: 1.2268 - val_loss: 1.1217 - val_categorical_crossentropy: 1.1217

Epoch 00001: val_loss improved from inf to 1.12168, saving model to keras_model
Epoch 2/8
636/636 [==============================] - 110s 173ms/step - loss: 1.0920 - categorical_crossentropy: 1.0920 - val_loss: 1.0072 - val_categorical_crossentropy: 1.0072

Epoch 00002: val_loss improved from 1.12168 to 1.00724, saving model to keras_model
Epoch 3/8
636/636 [==============================] - 109s 172ms/step - loss: 1.0139 - categorical_crossentropy: 1.0139 - val_loss: 0.9772 - val_categorical_crossentropy: 0.9772

Epoch 00003: val_loss improved from 1.00724 to 0.97720, saving model to keras_model
Epoch 4/8
636/636 [==============================] - 111s 174ms/step - loss: 0.8914 - categorical_crossentropy: 0.8914 - val_loss: 0.9249 - val_categorical_crossentropy: 0.

In [24]:
model = load_model('keras_model')

In [56]:
probas = model.predict(X_test)

In [54]:
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix

pred_indices = np.argmax(probas, axis=1)
classes = ['commenting','questioning','denying','supporting']
preds = classes[pred_indices]
print('Log loss: {}'.format(log_loss(classes[np.argmax(Y_test, axis=1)], probas)))
print('Accuracy: {}'.format(accuracy_score(classes[np.argmax(Y_test, axis=1)], preds)))

TypeError: only integer scalar arrays can be converted to a scalar index

In [41]:
# https://www.kaggle.com/reiinakano/basic-nlp-bag-of-words-tf-idf-word2vec-lstm

In [78]:
Xtest = tokenizer.texts_to_sequences(test['lemmatized'].values)
Xtest = pad_sequences(Xtest,maxlen=2000)
probas = model.predict(Xtest)

In [76]:
submission_df = pd.DataFrame(probas, columns=['commenting','questioning','denying','supporting'])

In [85]:
submission_df.shape[0]

199

In [83]:
print(model.predict(pad_sequences(tokenizer.texts_to_sequences('hello'),maxlen=2000)))

[[0.464029   0.10046138 0.2280001  0.20750946]
 [0.30604276 0.1502892  0.23522674 0.30844125]
 [0.464029   0.10046138 0.2280001  0.20750946]
 [0.464029   0.10046138 0.2280001  0.20750946]
 [0.464029   0.10046138 0.2280001  0.20750946]]
